# Imports

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

import copy
import functools
import gc
import itertools
import logging
import operator
import os
import pathlib
import re
import socket
import sys
import time
from collections import Counter
from dataclasses import asdict, dataclass, field
from enum import Enum
from functools import partial
from pathlib import Path
from pprint import PrettyPrinter, pprint
from typing import *

In [ ]:
%autoreload 2

import humanize
import matplotlib
import numpy as np
import pandas as pd
import scipy as sp
import tensorflow as tf
import yaml
from matplotlib import cm, patches, pyplot as plt
from numpy import ndarray
from numpy.random import RandomState
from progressbar import progressbar as pbar
from pymicro.file import file_utils
from sklearn import metrics, metrics as met, model_selection, preprocessing
from skimage import measure as skimage_measure
import tabulate
from tensorflow import keras
from tensorflow.keras import (
    callbacks as keras_callbacks,
    layers,
    losses,
    metrics as keras_metrics,
    optimizers,
    utils,
)
from tqdm import tqdm
from yaml import YAMLObject

In [ ]:
%autoreload 2

from tomo2seg import (
    analyse as tomo2seg_analyse,
    callbacks as tomo2seg_callbacks,
    data as tomo2seg_data,
    losses as tomo2seg_losses,
    schedule as tomo2seg_schedule,
    slack,
    slackme,
    utils as tomo2seg_utils,
    viz as tomo2seg_viz,
    volume_sequence,
)
from tomo2seg.data import EstimationVolume, Volume
from tomo2seg.logger import add_file_handler, dict2str, logger
from tomo2seg.model import Model as Tomo2SegModel

In [ ]:
# this registers a custom exception handler for the whole current notebook
get_ipython().set_custom_exc((Exception,), slackme.custom_exc)

# Args

In [ ]:
# [manual-input]


@dataclass
class Args:
    pass

from tomo2seg.datasets import (
    VOLUME_COMPOSITE_V1 as VOLUME_NAME_VERSION,
    VOLUME_COMPOSITE_V1_LABELS_REFINED3 as LABELS_VERSION,
#     VOLUME_FRACTURE00_SEGMENTED00 as VOLUME_NAME_VERSION,
#     VOLUME_FRACTURE00_SEGMENTED00_LABELS_REFINED3 as LABELS_VERSION,
)

volume_name, volume_version = VOLUME_NAME_VERSION
labels_version = LABELS_VERSION

random_state_seed = 42
runid = int(time.time())
# runid = 1607944057

parallel_nprocs = 30

logger.info(f"{volume_name=}")
logger.info(f"{volume_version=}")
logger.info(f"{labels_version=}")

In [ ]:
for f in os.listdir("../data"):
    if f"vol={volume_name}.{volume_version}" in f:
        print(f)

In [ ]:
# [manual-input]
estimation_volume_fullname = "vol=PA66GF30.v1.set=test.model=unet3d.vanilla03-f08.fold000.1606-842-005.runid=1607-521-380"

# Setup

In [ ]:
logger.setLevel(logging.DEBUG)
random_state = np.random.RandomState(random_state_seed)

In [ ]:
volume = Volume.with_check(
    name=volume_name, version=volume_version
)

logger.debug(f"volume=\n{dict2str(asdict(volume))}")

estimation_volume = EstimationVolume.from_fullname(
    estimation_volume_fullname
)

logger.debug(f"estimation_volume=\n{dict2str(asdict(estimation_volume))}")

In [ ]:
exec_name = f"{estimation_volume.fullname}.estimation-analysis"
exec_dir = estimation_volume.dir 
figs_dir = exec_dir

logger.info(f"{exec_name=}")
logger.info(f"{exec_dir=}")

In [ ]:
if estimation_volume.partition is None:
    raise NotImplementedError(f"{estimation_volume.partition=}")

# Setup

# Load data

In [ ]:
logger.info("Loading data from disk.")

data_volume = file_utils.HST_read(
    str(volume.data_path),  # it doesn't accept paths...
    
    autoparse_filename=False,  # the file names are not properly formatted
    data_type=volume.metadata.dtype,
    dims=volume.metadata.dimensions,
    verbose=False,
)

logger.debug(f"{data_volume.shape=}")

if estimation_volume.partition is not None:
    
    logger.info(f"Cutting data {estimation_volume.partition}.")
    
    data_volume = estimation_volume.partition.get_volume_partition(data_volume)
    
    logger.debug(f"{data_volume.shape=}")

In [ ]:
logger.info("Loading labels from disk.")

labels_volume = file_utils.HST_read(
    str(volume.versioned_labels_path(labels_version)),  # it doesn't accept paths...
    
    autoparse_filename=False,  # the file names are not properly formatted
    data_type="uint8",
    dims=volume.metadata.dimensions,
    verbose=False,
)

logger.debug(f"{labels_volume.shape=}")

if estimation_volume.partition is not None:
    
    logger.info(f"Cutting labels {estimation_volume.partition}.")
    
    labels_volume = estimation_volume.partition.get_volume_partition(labels_volume)
    
    logger.debug(f"{labels_volume.shape=}")

In [ ]:
logger.info("Loading predictions from disk.")

predictions_volume = file_utils.HST_read(
    str(estimation_volume.predictions_path),  # it doesn't accept paths...
    
    autoparse_filename=False,  # the file names are not properly formatted
    data_type="uint8",
    dims=estimation_volume.partition.shape if estimation_volume.partition is not None else volume.metadata.dimensions,
    verbose=False,
)

logger.debug(f"{predictions_volume.shape=}")

In [ ]:
logger.info("Loading probabilities from disk.")

if estimation_volume.probabilities_path.exists():
    
    proba_dtype = np.float16
    
    # float16 instead of 64 to save memory
    probas_volume = np.load(estimation_volume.probabilities_path).astype(proba_dtype)
    
    logger.debug(f"{probas_volume.shape=}")
    
    probabilities_are_available = True
    
else:
    logger.warning("Probabilities are not available.")
    probabilities_are_available = False

# useful variables

In [ ]:
labels_idx = volume.metadata.labels
labels_names = [volume.metadata.labels_names[idx] for idx in labels_idx]

labels_idx_name = list(zip(labels_idx, labels_names))

n_classes = len(labels_idx)

logger.debug(f"{n_classes=}")
logger.debug(f"{labels_idx=}")
logger.debug(f"{labels_names=}")

# [compute] confusion volume

In [ ]:
labels_indices = volume.metadata.labels

In [ ]:
if n_classes > 100:
    raise NotImplementedError(f"{n_classes=}")

logger.info("Computing confusion volume encoding.")

cv_dtype = np.int16

# cv = confusion volume
cv_encoding = {
    # (gt, pred)
    (gt_idx, pred_idx): 100 * gt_idx + pred_idx
    for gt_idx, pred_idx in itertools.product(labels_indices, labels_indices)
}

logger.debug(f"cv_encoding=\n{dict2str(cv_encoding)}")
estimation_volume["cv_encoding"] = cv_encoding

cv_encoding_inv = dict(map(lambda x: tuple(reversed(x)), cv_encoding.items()))

logger.debug(f"cv_encoding_inv=\n{dict2str(cv_encoding_inv)}")
estimation_volume["cv_encoding_inv"] = cv_encoding_inv

In [ ]:
logger.info("Computing confusion volume.")

# 10000 is an impossible encoding
conf_vol = np.full_like(labels_volume, 10000, dtype=cv_dtype)

for (gt_idx, pred_idx), encoded_value in pbar(
    cv_encoding.items(),
    max_value=len(cv_encoding)
):

    conf_vol[
        (labels_volume == gt_idx) & (predictions_volume == pred_idx)
    ] = encoded_value

In [ ]:
assert np.all(conf_vol != 10000)

# [save] confusion volume

In [ ]:
logger.info(f"Saving confusion volume.")
logger.debug(f"{estimation_volume.confusion_volume_path=}")

file_utils.HST_write(conf_vol, str(estimation_volume.confusion_volume_path))    

# [compute] error volume

In [ ]:
error_volume = np.full_like(labels_volume, False, dtype=bool)

for label_idx in pbar(labels_idx):
    
    encoded_value = cv_encoding[(label_idx, label_idx)]
    
    logger.debug(f"{label_idx=} {encoded_value=}")
    
    error_volume |= (conf_vol == encoded_value)
    
error_volume = ~error_volume

# [save] error volume 

In [ ]:
logger.info(f"Saving error volume.")
logger.debug(f"{estimation_volume.error_volume_path=}")

file_utils.HST_write(error_volume, str(estimation_volume.error_volume_path))    

# [compute] confusion matrix

In [ ]:
max_encoded_val = max(cv_encoding.values())

logger.debug(f"{max_encoded_val=}")

# cm = confusion matrix
cm_encoded_counts = np.bincount(conf_vol.ravel(), minlength=max_encoded_val + 1)

cm_counts = {}

for gt_pred_indices, enc_val in cv_encoding.items():
    
    cm_counts[gt_pred_indices] = cm_encoded_counts[enc_val]

In [ ]:
conf_matrix = [
    [
        cm_counts[(gt_idx, pred_idx)]
        for pred_idx in labels_indices
    ]
    for gt_idx in labels_indices
]

conf_matrix = np.array(conf_matrix)

# [save] confusion matrix

In [ ]:
logger.info(f"Saving confusion matrix.")
logger.debug(f"{estimation_volume.confusion_matrix_path=}")

estimation_volume["confusion_matrix_dtype"] = conf_matrix.dtype

np.save(estimation_volume.confusion_matrix_path, conf_matrix)

In [ ]:
assert (ncorrect_error_volume := (~error_volume).sum()) == (ncorrect_conf_matrix := conf_matrix.diagonal().sum()), (
    f"{ncorrect_error_volume=} {ncorrect_conf_matrix=}"
)

# [compute-save] roc curve

In [ ]:
logger.info("Computing and saving ROC curves")

roc_dfs = []

for label_idx in pbar(labels_idx):
    
    logger.debug(f"Computing roc curve {label_idx=}")
    
    fpr, tpr, th = metrics.roc_curve(
        labels_volume.ravel(), 
        probas_volume[:, :, :, label_idx].ravel(), 
        
        pos_label=label_idx,
        drop_intermediate=True
    )
    
    roc_df = pd.DataFrame(
        data={
            "fpr": fpr,
            "tpr": tpr,
            "th": th,
        }
    ).T
    
    logger.debug(f"{label_idx=} {roc_df.shape=}")
    
    roc_path = estimation_volume.get_roc_curve_csv_path(label_idx)

    logger.debug(f"Saving roc curve of {label_idx=} at {roc_path=}")
    
    roc_df.to_csv(
        roc_path,
        header=True,
        index=True,
    )
    
    roc_dfs.append(roc_df)

# [compute] multi-class roc-auc

In [ ]:
raveled_probas = probas_volume.reshape(-1, n_classes)
raveled_probas = raveled_probas / raveled_probas.sum(axis=-1, keepdims=True)  # more numerically precise...

In [ ]:
multiclass_roc_auc_macro_ovr = tomo2seg_analyse.multiclass_roc_auc_score(
    y_true=labels_volume.ravel(),
    y_score=raveled_probas,
    average="macro",
    multi_class="ovr",
    labels=labels_idx,
)

logger.debug(f"{multiclass_roc_auc_macro_ovr=}")

In [ ]:
multiclass_roc_auc_macro_ovo = tomo2seg_analyse.multiclass_roc_auc_score(
    y_true=labels_volume.ravel(),
    y_score=raveled_probas,
    average="macro",
    multi_class="ovo",
    labels=labels_idx,
)

logger.debug(f"{multiclass_roc_auc_macro_ovo=}")

# [compute] 2d error blobs

In [ ]:
logger.info("Computing 2d error blobs in the 3 directions.")

In [ ]:
xy_blob_props = tomo2seg_analyse.get_slice_props_parallel(error_volume, data_volume, 2, nprocs=parallel_nprocs) 

In [ ]:
xz_blob_props = tomo2seg_analyse.get_slice_props_parallel(error_volume, data_volume, 1, nprocs=parallel_nprocs)

In [ ]:
yz_blob_props = tomo2seg_analyse.get_slice_props_parallel(error_volume, data_volume, 0, nprocs=parallel_nprocs)

In [ ]:
logger.info("Converting 2d blob props dicts to data frames.")

all_blob_props = [
    yz_blob_props,  # normal axis x (0)
    xz_blob_props,  #         ... y (1)
    xy_blob_props,  #         ... z (2)
]

for axis in range(3):
    
    logger.debug(f"{axis=}")
    
    blob_props = all_blob_props[axis]
    
    ref_shape = len(blob_props["area"])
    
    for k in blob_props.keys():
        assert (shap := len(blob_props[k])) == ref_shape, f"{k=} {shap=} {ref_shape=}"
    
    all_blob_props[axis] = pd.DataFrame(blob_props)
    
    logger.debug(f"{all_blob_props[axis].shape=}")


In [ ]:
error_2dblobs_props = pd.concat(
    all_blob_props, 
    axis=0,
)

# [save] 2d error blobs

In [ ]:
logger.info("Saving 2d error blobs.")

filepath = estimation_volume.error_2dblobs_props_path

logger.debug(f"{filepath=}")

error_2dblobs_props.to_csv(filepath, index=False)

# [compute] 3d error blobs

# [save] 3d error blobs

# derived computations

## classification report

In [ ]:
report_dict = tomo2seg_analyse.get_classification_report(
    cm=conf_matrix,
    rocs=tuple(
        {
            "tpr": roc_df.loc["tpr"].values,
            "fpr": roc_df.loc["fpr"].values,
        }
        for roc_df in roc_dfs
    )
)

report_dict["macro"]["multiclass-roc-auc-ovr"] = multiclass_roc_auc_macro_ovr
report_dict["macro"]["multiclass-roc-auc-ovo"] = multiclass_roc_auc_macro_ovo

for idx, name in labels_idx_name:
    report_dict[name] = report_dict[idx]
    del report_dict[idx]

In [ ]:
yaml_dump = functools.partial(
    yaml.dump,
    default_flow_style=False, 
    indent=4, 
    sort_keys=False
)

logger.info(
    f"Saving exact classification report."
    f"{estimation_volume.voxelwise_classification_report_exact=}"
)

logger.info(
    f"Saving humanized classification report."
    f"\n{estimation_volume.voxelwise_classification_report_human=}"
)

with estimation_volume.voxelwise_classification_report_exact.open('w') as f:
    yaml_dump(report_dict, f)

with estimation_volume.voxelwise_classification_report_human.open('w') as f:
    humanized_report_str = yaml_dump(
        report_dict, 
        Dumper=tomo2seg_analyse.ClassifReportHumandDumper,
    )
    f.write(humanized_report_str)

In [ ]:
logger.info(f"classification report=\n{humanized_report_str}")

# plots

## classification report (table)

In [ ]:
table = []
table_human_simple = []
table_human_detail = []

cols0 = ["class/average"]

cols1 = [
    "accuracy",
    "precision",
    "recall",
    "f1",
    "roc-auc",
    "jaccard",
]

cols2 = [
    "tp",
    "fp",
    "fn",
    "tn",
]

cols3 = [
    "support",
    "npred",
]

cols = cols0 + cols1 + cols2 + cols3

for key in labels_names + ["macro", "micro"]:
    
    dic = report_dict[key]
        
    line = [key] + [
        dic.get(m, None)
        for m in cols1 + cols2 + cols3
    ]
    
    line_human_simple = [
        v if isinstance(v, str) else
        (
            f"{humanize.intword(v)}" + (
                f" ({dic[col_rel]:.1%})" if col in ("tp", "fp", "fn", "tn") and (col_rel := col + '_relative') in dic else ""
            )
        ) if isinstance(v, int) else 
        f"{v:.1%}" if v is not None else 
        "-"
        for v, col in zip(line, cols)
    ]
    
    line_human_detail = [
        v if isinstance(v, str) else
        (
            f"{humanize.intcomma(v)} ({humanize.intword(v)})" + (
                f" ({dic[col_rel]:.4%})" if col in ("tp", "fp", "fn", "tn") and (col_rel := col + '_relative') in dic else ""
            )
        ) if isinstance(v, int) else 
        f"{v:.4%}" if v is not None else 
        "-"
        for v, col in zip(line, cols)
    ]


    table.append(line)
    table_human_simple.append(line_human_simple)
    table_human_detail.append(line_human_detail)
    
table_human_simple.insert(-2, [])
table_human_detail.insert(-2, [])

df = pd.DataFrame(table, columns=cols).set_index(cols0[0])

logger.debug(f"{estimation_volume.classification_report_table_exact_csv_path=}")

df.to_csv(
    estimation_volume.classification_report_table_exact_csv_path, 
    header=True,
    index=True,
)

table_str = tabulate.tabulate(table_human_simple, headers=cols)
logger.info(table_str)

logger.debug(f"{estimation_volume.classification_report_table_human_simple_txt_path=}")

with estimation_volume.classification_report_table_human_simple_txt_path.open("w") as f:
    f.write(table_str)

table_str = tabulate.tabulate(table_human_detail, headers=cols)
logger.info(table_str)

logger.debug(f"{estimation_volume.classification_report_table_human_detail_txt_path=}")

with estimation_volume.classification_report_table_human_detail_txt_path.open("w") as f:
    f.write(table_str)
    

## confusion matrix

In [ ]:
logger.debug(f"Saving figure {(fig_name := 'confusion-matrix.png')=}")

fig, axs = plt.subplots(
    n_rows := 2, 
    n_cols := 2, 
    figsize=(n_cols * (sz := 7), n_rows * sz), 
    dpi=(dpi := 120),
    gridspec_kw=dict(wspace=sz/30),
)

cm_display = tomo2seg_viz.ConfusionMatrixDisplay(
    cm_normalized := conf_matrix, 
    display_labels=labels_names,
).plot(
    values_format=None, 
    cmap=cm.inferno, 
    ax=axs[0, 0],
    cmap_vmax=int(conf_matrix.max()),
)

cm_display.ax_.set_title("Confusion matrix")

cm_display = tomo2seg_viz.ConfusionMatrixDisplay(
    cm_normalized := conf_matrix / conf_matrix.sum(), 
    display_labels=labels_names,
).plot(
    values_format='.2%', 
    cmap=cm.inferno, 
    ax=axs[0, 1]
)

cm_display.ax_.set_title("Confusion matrix (normalized)")

cm_display = tomo2seg_viz.ConfusionMatrixDisplay(
    cm_true_label_normalized := conf_matrix / conf_matrix.sum(axis=1).reshape(-1, 1), 
    display_labels=labels_names,
).plot(
    values_format='.1%', 
    cmap=cm.inferno, 
    ax=axs[1, 0],
)
cm_display.ax_.set_title("Confusion matrix normalized (%)\nby *true label* (by line)\ndiagonal = recall")

cm_display = tomo2seg_viz.ConfusionMatrixDisplay(
    cm_predicted_label_normalized := conf_matrix / conf_matrix.sum(axis=0).reshape(1, -1), 
    display_labels=labels_names,
).plot(
    values_format='.1%', 
    cmap=cm.inferno, 
    ax=axs[1, 1],
)

cm_display.ax_.set_title("Confusion matrix normalized (%)\nby *predicted label* (by column)\ndiagonal = precision")

fig.suptitle(f"Confusion matrices {estimation_volume_fullname}");

fig.savefig(
    fname=figs_dir / fig_name,
    dpi=dpi,
    metadata={"Title": f"vol={volume.fullname}::analysis::{fig_name}",}
)

## roc curve

In [ ]:
logger.info(f"Saving figure of ROC curves at {(fig_name := f'roc-curves.png')}")

fig, axs = plt.subplots(
    n_rows := 1, 
    n_cols := 2, 
    figsize=(n_cols * (sz := 7), n_rows * sz), 
    dpi=(dpi := 130),
)

# [manual-input]
zoom = np.array(((0, .15), (.85, 1)))

fig.suptitle("Per class ROC curves")

ax_full, ax_zoom = axs[0], axs[1]
ax_full.set_title("Full curve range [0, 1] x [0, 1]")
ax_full.set_xlim(0, 1)
ax_full.set_ylim(0, 1)

ax_zoom.set_title(f"Zoom on [{zoom[0, 0]}, {zoom[0, 1]}] x [{zoom[1, 0]}, {zoom[1, 1]}]")
ax_zoom.set_xlim(*zoom[0])
ax_zoom.set_ylim(*zoom[1])

for label_idx, roc_df in zip(labels_idx, roc_dfs):
    
    fpr = roc_df.loc["fpr"].values
    tpr = roc_df.loc["tpr"].values
    
    roc_display = metrics.RocCurveDisplay(
        fpr=fpr, 
        tpr=tpr, 
        estimator_name=f"{label_idx}",
    )
    
    for ax in axs:
        roc_display.plot(ax=ax)  

max_label_name_length = max(*map(len, labels_names))

for label_idx, roc_df in zip(labels_idx, roc_dfs):
    
    fpr = roc_df.loc["fpr"].values
    tpr = roc_df.loc["tpr"].values
    
    label_name = labels_names[label_idx]
    
    ax_full.get_legend().texts[label_idx].set_text(
        label_name.ljust(max_label_name_length) +
        f"AUC={report_dict[label_name]['roc-auc']:.2%}"
    )

    
ax_zoom.legend_ = None

## volumetric fraction

In [ ]:
class_proportion_gt = conf_matrix.sum(axis=1)
class_proportion_pred = conf_matrix.sum(axis=0)

In [ ]:
fig, ax = plt.subplots(
    nrows := 1, ncols := 2, 
    figsize=(ncols * (sz := 7), nrows * sz), 
    dpi=(dpi := 90), 
    gridspec_kw=dict(wspace=sz/16, hspace=sz/12)
)

display_gt = tomo2seg_viz.ClassImbalanceDisplay(
    volume_name=f"{volume.fullname} (partition={estimation_volume.partition.alias})",
    labels_idx=labels_idx,
    labels_names=labels_names,
    labels_counts=class_proportion_gt.tolist(),
).plot(ax[0])


model_name_idx = estimation_volume.fullname.index("model")

display_pred = tomo2seg_viz.ClassImbalanceDisplay(
    volume_name=f"{estimation_volume.fullname[:model_name_idx-1]}\n{estimation_volume.fullname[model_name_idx:]}",
    labels_idx=labels_idx,
    labels_names=labels_names,
    labels_counts=class_proportion_pred.tolist(),
).plot(ax[1])

fig.suptitle(f"Volumetric fraction comparison.")

logger.info(f"Saving figure {(figname := 'volumetric-fraction.png')=}")

fig.savefig(
    fname=figs_dir / figname,
    format="png",
)

# Physical metrics

# Notable slices

In [ ]:
logger.info(f"Finding notable slices.")

In [ ]:
error_2dblobs_props = pd.read_csv(estimation_volume.error_2dblobs_props_path)

MIN_ERROR_BLOB_AREA = 1

logger.info(f'filtering error blobs < {MIN_ERROR_BLOB_AREA=}')

logger.debug(f"before {(nblobs := error_2dblobs_props.shape[0])=} ({humanize.intcomma(nblobs)})")

error_2dblobs_props = error_2dblobs_props[error_2dblobs_props.area > MIN_ERROR_BLOB_AREA]

logger.debug(f"after {(nblobs := error_2dblobs_props.shape[0])=} ({humanize.intcomma(nblobs)})")

error_2dblobs_props = error_2dblobs_props.set_index(["normal_axis"])

## `add_notable_slices`

In [ ]:
notable_slices = {}

def add_notable_slices(func: Callable[[pd.DataFrame], Tuple[dict, dict]], axes=(0, 1, 2)):
    for axis in axes:
        
        name, slice_idx, custom_attrs = func(error_2dblobs_props.loc[axis])

        name += f".{axis=}"

        notable_slice_dict = notable_slices[name] = {
            "name": name,
            "normal_axis": axis,
            "slice_idx": int(slice_idx),
        }
        
        slice_ = [slice(None), slice(None), slice(None)]
        slice_[axis] = slice(slice_idx, slice_idx + 1)
        slice_ = tuple(slice_)
        
        notable_slice_dict.update({
            "slice": slice_,
            **custom_attrs,
        })
        

def add_notable_slices_blobwise(func: Callable[[pd.DataFrame], Tuple[dict, dict]], axes=(0, 1, 2)):
    for axis in axes:
        
        name, row, custom_attrs = func(error_2dblobs_props.loc[axis])

        name += f".{axis=}"

        notable_slice_dict = notable_slices[name] = {
            "name": name,
            "normal_axis": axis,
            "slice_idx": int(row.slice_idx),
        }

        centroid3d = tuple(
            int(row[f"centroid-{axx}"])
            for axx in range(3)
        )

        centroid2d = tuple(
            val 
            for axx, val in enumerate(centroid3d)
            if axx != axis
        )

        bbox3d = (
            slice(int(row[f"bbox-0"]), int(row[f"bbox-3"])),
            slice(int(row[f"bbox-1"]), int(row[f"bbox-4"])),
            slice(int(row[f"bbox-2"]), int(row[f"bbox-5"])),
        )

        bbox2d = tuple(
            val 
            for axx, val in enumerate(bbox3d)
            if axx != axis
        )

        notable_slice_dict.update({
            "centroid3d": centroid3d,
            "centroid2d": centroid2d,
            "bbox3d": bbox3d,
            "bbox2d": bbox2d,
            **custom_attrs,
        })

## blobwise criterion

### error blob max area

In [ ]:
def max_area(df):
    row = df.iloc[df.area.argmax()]
    custom_attrs = {"blob-area": int(row["area"])}
    return "error-blob.max-area", row, custom_attrs

add_notable_slices_blobwise(max_area)

### error blob max bbox.shape[i] for i in  {0, 1, 2}

In [ ]:
def max_bbox_shape(df, dim):
    
    bbox_shape_dim = df[f"bbox-{dim + 3}"] - df[f"bbox-{dim}"] 
    
    arg_max = bbox_shape_dim.argmax()
    row = df.iloc[arg_max]
    
    custom_attrs = {f"blob-bbox.length.axis={dim}": int(bbox_shape_dim.iloc[arg_max])}
    
    return f"error-blob.max-bbox-lenghth-axis={axis}", row, custom_attrs

add_notable_slices_blobwise(partial(max_bbox_shape, dim=1), axes=(0,))
add_notable_slices_blobwise(partial(max_bbox_shape, dim=2), axes=(0,))
add_notable_slices_blobwise(partial(max_bbox_shape, dim=0), axes=(1,))
add_notable_slices_blobwise(partial(max_bbox_shape, dim=2), axes=(1,))
add_notable_slices_blobwise(partial(max_bbox_shape, dim=0), axes=(2,))
add_notable_slices_blobwise(partial(max_bbox_shape, dim=1), axes=(2,))

### error blob max `major_axis_length`

In [ ]:
def max_major_axis_length(df):
    row = df.iloc[df.major_axis_length.argmax()]
    custom_attrs = {"blob-major-axis-length": float(row["major_axis_length"])}
    return "error-blob.max-major-axis-length", row, custom_attrs

add_notable_slices_blobwise(max_major_axis_length)

### error blob max `minor_axis_length`

In [ ]:
def max_minor_axis_length(df):
    row = df.iloc[df.minor_axis_length.argmax()]
    custom_attrs = {"blob-minor-axis-length": float(row["minor_axis_length"])}
    return "error-blob.max-minor-axis-length", row, custom_attrs

add_notable_slices_blobwise(max_minor_axis_length)

## max error area

In [ ]:
def max_error_area(df):
    area_per_slice = df[["area", "slice_idx"]].groupby("slice_idx").sum()
    slice_idx = area_per_slice.area.argmax()
    custom_attrs = {"slice-error-area": int(area_per_slice.loc[slice_idx].area)}
    return "max-error-area", slice_idx, custom_attrs

add_notable_slices(max_error_area)

## max error blob avg area

In [ ]:
def max_error_blob_avg_area(df):
    avg_blob_area_per_slice = df[["area", "slice_idx"]].groupby("slice_idx").mean()
    slice_idx = avg_blob_area_per_slice.area.argmax()
    custom_attrs = {"slice-avg-error-blob-area": int(avg_blob_area_per_slice.loc[slice_idx].area)}
    return "max-avg-error-blob-area", slice_idx, custom_attrs

add_notable_slices(max_error_blob_avg_area)

# [save] notable slices

In [ ]:
logger.info(f"{dict2str(notable_slices)}")

estimation_volume["notable-slices"] = notable_slices

# Save notebook

In [ ]:
this_nb_name = "analyse-estimation-00-krilin92.ipynb"

import os
this_dir = os.getcwd()
logger.warning(f"{this_nb_name=} {this_dir=}")

os.system(f"jupyter nbconvert {this_dir}/{this_nb_name} --output-dir {str(exec_dir)} --to html")

# End

In [ ]:
slack.notify("end")